# ElasticSearch BM25

>[Elasticsearch](https://www.elastic.co/elasticsearch/) is a distributed, RESTful search and analytics engine. It provides a distributed, multitenant-capable full-text search engine with an HTTP web interface and schema-free JSON documents.

>In information retrieval, [Okapi BM25](https://en.wikipedia.org/wiki/Okapi_BM25) (BM is an abbreviation of best matching) is a ranking function used by search engines to estimate the relevance of documents to a given search query. It is based on the probabilistic retrieval framework developed in the 1970s and 1980s by Stephen E. Robertson, Karen Spärck Jones, and others.

>The name of the actual ranking function is BM25. The fuller name, Okapi BM25, includes the name of the first system to use it, which was the Okapi information retrieval system, implemented at London's City University in the 1980s and 1990s. BM25 and its newer variants, e.g. BM25F (a version of BM25 that can take document structure and anchor text into account), represent TF-IDF-like retrieval functions used in document retrieval.

This notebook shows how to use a retriever that uses `ElasticSearch` and `BM25`.

For more information on the details of BM25 see [this blog post](https://www.elastic.co/blog/practical-bm25-part-2-the-bm25-algorithm-and-its-variables).

In [ ]:
%pip install --upgrade --quiet  elasticsearch

In [1]:
from langchain_community.retrievers import (
    ElasticSearchBM25Retriever,
)

Ensure your elasticsearch instance is setup.

In [ ]:
import os

elastic_endpoint =os.getenv("ELASTIC_ENDPOINT")
api_key = os.getenv("ELASTIC_API_KEY")

## Create New Retriever

In [2]:
# Example of a retriever
elasticsearch_url = "http://localhost:9200"
retriever = ElasticSearchBM25Retriever.create(elasticsearch_url, 
                                              index_name="langchain-index-4",
                                              delete_if_exists=False,
                                              analyzer_type="standard",
                                              es_params={"api_key": api_key})

You could set `delete_if_exists=True` in case you want to recreate the index. You could also set an analyzer for the index (default is the standard analyzer).

In [3]:
# Alternatively, you can load an existing index
# import elasticsearch
# elasticsearch_url="http://localhost:9200"
# retriever = ElasticSearchBM25Retriever(elasticsearch.Elasticsearch(elasticsearch_url), "langchain-index")

## Add texts (if necessary)

We can optionally add texts to the retriever (if they aren't already in there)

In [4]:
retriever.add_texts(["foo", "bar", "world", "hello", "foo bar"])

['622aa8f4-9e2f-4b13-ae85-392a72039bed',
 'bb156db7-1a2a-4839-8e92-3057a281e931',
 '4168767e-27eb-472a-a1e5-287edb6d7a07',
 '2f99426b-99d5-4227-8d83-8ac1acde505d',
 '68e47327-c505-4cd8-b5ad-1c67836fb773']

Alternatively, documents can be added

In [8]:
from langchain_core.documents import Document

docs = [
    Document(
        metadata= {"title": "The Anonymous Cat's Fame"},
        page_content="In a world where anonymity reigns, a mysterious cat wanders the streets, unaware of its own fame among the locals."
    ),
    Document(
        metadata= {"title": "The Secret Chocolate Chip Cookie Recipe"},
        page_content="The secret recipe for the world's best chocolate chip cookie remains hidden, as the baker took it to the grave, leaving only crumbs of information behind."
    ),
    Document(
        metadata= {"title": "The Mysterious Mural Artist"},
        page_content="An unidentified artist creates stunning murals throughout the city, their identity shrouded in mystery, as they continue to inspire and captivate the public."
    ),
    Document(
        metadata= {"title": "The Lost Letter's Heartwarming Tale"},
        page_content="A lost letter, written by an anonymous author, reveals a heartwarming tale of love and sacrifice, discovered and cherished by a stranger."
    ),
    Document(
        metadata= {"title": "The Forgotten Era's Secrets"},
        page_content="In the depths of the library, an ancient book, devoid of any identifying marks, unveils secrets and wisdom from a forgotten era."
    )
]

retriever.add_documents(docs)

['fc77d1fe-2865-45b1-b561-8948dbe12c77',
 'da942234-3ad5-462a-9c86-4962d5585303',
 'e77ff15c-64b0-4bf9-85d3-93a9d40d67bf',
 '0218eb3b-76b9-422c-a7c5-750725f5311c',
 '5246f9e6-e9a7-48d4-9964-1d403d7b5f58']

## Use Retriever

We can now use the retriever!

In [9]:
result = retriever.invoke("chocoloate chip")

In [10]:
result

[Document(metadata={'title': 'The Secret Chocolate Chip Cookie Recipe'}, page_content="The secret recipe for the world's best chocolate chip cookie remains hidden, as the baker took it to the grave, leaving only crumbs of information behind.")]